In [1]:
import keras 

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
import cv2
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras import optimizers
from keras.datasets import mnist
from keras.utils import to_categorical

Reshaping cells

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


Threshold for removing noise

In [5]:
_,X_train_th = cv2.threshold(X_train,127,255,cv2.THRESH_BINARY)
_,X_test_th = cv2.threshold(X_test,127,255,cv2.THRESH_BINARY)


In [6]:
X_train = X_train_th.reshape(-1,28,28,1)
X_test = X_test_th.reshape(-1,28,28,1)

In [7]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [8]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


In [9]:
input_shape = (28,28,1)
number_of_classes = 10

In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_of_classes, activation='softmax'))



Final Showdown -- compiling

In [13]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [15]:
history = model.fit(X_train, y_train,epochs=5, shuffle=True,
                    batch_size = 200,validation_data= (X_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 105s 2ms/step - loss: 1.7563 - accuracy: 0.7651 - val_loss: 0.1140 - val_accuracy: 0.9655
Epoch 2/5
60000/60000 [==============================] - 108s 2ms/step - loss: 0.2018 - accuracy: 0.9429 - val_loss: 0.0710 - val_accuracy: 0.9773
Epoch 3/5
60000/60000 [==============================] - 99s 2ms/step - loss: 0.1280 - accuracy: 0.9631 - val_loss: 0.0578 - val_accuracy: 0.9826
Epoch 4/5
60000/60000 [==============================] - 93s 2ms/step - loss: 0.1004 - accuracy: 0.9717 - val_loss: 0.0502 - val_accuracy: 0.9852
Epoch 5/5
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0778 - accuracy: 0.9770 - val_loss: 0.0570 - val_accuracy: 0.9831


In [16]:
validationLoss, validationAccuracy = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 4s 436us/step


In [18]:
print(validationLoss, "|||", validationAccuracy)

0.05701461560284151 ||| 0.9830999970436096


In [93]:
%store -r extracted_digits

In [99]:
import cv2
import numpy as np
present = 0
from tensorflow.keras.preprocessing.image import img_to_array
board = np.zeros((9,9), dtype='int')
for i in range(81):
    if extracted_digits[i] is not None:
        present+=1
        value = cv2.resize(extracted_digits[i], (28,28))
#         value = value.astype("float") / 255.00
        value = img_to_array(value)
        value = np.expand_dims(value, axis=0)
        prediction = model.predict(value) 
        board[i // 9][i%9] = np.argmax(prediction)
print( present, "number of cells which are non-empty")

31 number of cells which are non-empty


In [100]:
for i in range(9):
    for j in range(9):
        print(board[i][j], end = " ")
    print('\n') 

0 0 4 1 9 9 0 7 9 

1 0 0 0 0 9 0 9 0 

0 0 9 0 0 0 4 0 0 

0 0 6 7 0 0 0 9 0 

1 0 1 0 0 0 2 0 0 

0 1 0 0 0 6 0 0 0 

0 0 6 0 0 0 7 0 0 

0 6 0 6 0 0 0 0 6 

6 0 0 5 6 7 3 0 0 

